In [1]:
from pyspark.sql import SparkSession

# Spark-Session starten
spark = SparkSession.builder \
    .appName("ShipRouteOptimization") \
    .getOrCreate()

# Optional: Log-Level reduzieren für weniger Output
spark.sparkContext.setLogLevel("WARN")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/15 11:55:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
import requests
from io import BytesIO
import pandas as pd
import zipfile
from concurrent.futures import ThreadPoolExecutor
import folium

# Schritt 1: Funktion zum Herunterladen und Entpacken von ZIP-Archiv und Laden der CSV
def download_and_unzip_csv(url):
    response = requests.get(url)
    response.raise_for_status()
    zipfile_bytes = BytesIO(response.content)
    with zipfile.ZipFile(zipfile_bytes, 'r') as z:
        csv_filename = z.namelist()[0] 
        with z.open(csv_filename) as csv_file:
            return pd.read_csv(csv_file)

# Schritt 2: Liste der CSV (ZIP)-URLs
csv_urls = ["https://web.ais.dk/aisdata/aisdk-2024-03-01.zip", 
            "https://web.ais.dk/aisdata/aisdk-2024-03-02.zip"
           ]

# Schritt 3: Paralleles Herunterladen und Verarbeiten der ZIP-Dateien
with ThreadPoolExecutor(max_workers=10) as executor:
    dfs = list(executor.map(download_and_unzip_csv, csv_urls))

# Schritt 4: Alle DataFrames in einen großen DataFrame kombinieren
combined_df = pd.concat(dfs, ignore_index=True)

# Einige Beispielzeilen ausgeben um Mögliche MMSI-Nummern anzuzeigen
print(combined_df.head(10))

           # Timestamp Type of mobile       MMSI   Latitude  Longitude  \
0  01/03/2024 00:00:00        Class A  219000873  56.990910  10.304543   
1  01/03/2024 00:00:00   Base Station    2190068  56.447260  10.945872   
2  01/03/2024 00:00:00        Class A  219016683  56.800165   9.024933   
3  01/03/2024 00:00:00        Class A  219000615  56.967093   9.224287   
4  01/03/2024 00:00:00   Base Station    2190071  57.110043   8.648282   
5  01/03/2024 00:00:00        Class A  219017664  56.974950   8.922530   
6  01/03/2024 00:00:00        Class A  219002686  56.795143   8.863960   
7  01/03/2024 00:00:00        Class A  219030053  57.058252   9.900817   
8  01/03/2024 00:00:00        Class A  219670000  55.463782   8.444915   
9  01/03/2024 00:00:00        Class A  211417590  54.524345  12.675237   

          Navigational status  ROT  SOG    COG  Heading  ... Length  \
0      Under way using engine  NaN  0.0   30.2      NaN  ...    NaN   
1               Unknown value  NaN  NaN    

In [ ]:
########################################################
# 2. Basisstationen herausfiltern ("Type of mobile" != "Base Station")
########################################################

# Nur Zeilen behalten, die keine "Base Station" sind
if "Type of mobile" in combined_df.columns:
    combined_df = combined_df[combined_df["Type of mobile"] != "Base Station"]
else:
    print("Warnung: 'Type of mobile' Spalte nicht vorhanden, Überspringe diesen Schritt.")

In [14]:
########################################################
# 3. Nur relevante Spalten behalten, um Datenmenge zu reduzieren
########################################################

relevant_columns = ["MMSI", "Latitude", "Longitude", "# Timestamp"]
combined_df = combined_df[relevant_columns]

########################################################
# 4. Timestamp in datetime konvertieren
########################################################

combined_df['# Timestamp'] = pd.to_datetime(combined_df['# Timestamp'], format='%d/%m/%Y %H:%M:%S', errors='coerce')

########################################################
# 5. Filtern von MMSI-Nummern, die genug Datenpunkte haben
########################################################

# Anzahl Datenpunkte pro MMSI bestimmen
mmsi_counts = combined_df.groupby("MMSI").size().reset_index(name="count")

# Schwelle definieren (z.B. mindestens 50 Punkte)
threshold = 50
valid_mmsi = mmsi_counts[mmsi_counts["count"] >= threshold]["MMSI"].unique()

# Gefilterter DataFrame nur mit MMSI, die genügend Datenpunkte haben
filtered_by_count_df = combined_df[combined_df["MMSI"].isin(valid_mmsi)]

########################################################
# 6. Nach bestimmter MMSI und mehreren Datums filtern + Route plotten
########################################################

mmsi_number = 246846000  # Ersetze mit deiner MMSI
desired_dates = ["2024-03-01", "2024-03-02"]  # Beispiel: zwei unterschiedliche Tage

# Prüfen, ob MMSI genug Daten hat
if mmsi_number not in valid_mmsi:
    print(f"MMSI {mmsi_number} hat nicht genügend Datenpunkte, um eine aussagekräftige Route anzuzeigen.")
else:
    # Liste von datetime.date Objekten aus den Strings erstellen
    date_objects = [pd.to_datetime(d).date() for d in desired_dates]
    
    # Nach MMSI und Datum filtern
    route_df = filtered_by_count_df[
        (filtered_by_count_df["MMSI"] == mmsi_number) &
        (filtered_by_count_df["# Timestamp"].dt.date.isin(date_objects))
    ].sort_values("# Timestamp")

    # Überprüfen, ob gefilterte Daten vorhanden sind
    if route_df.empty:
        print(f"Keine Daten für MMSI {mmsi_number} an den angegebenen Datumswerten: {desired_dates}")
    else:
        # Karte erstellen und Route plotten
        mean_lat = route_df["Latitude"].mean()
        mean_lon = route_df["Longitude"].mean()
        
        route_map = folium.Map(location=[mean_lat, mean_lon], zoom_start=8)
        
        # Koordinatenliste für PolyLine
        coords = route_df[["Latitude", "Longitude"]].values.tolist()
        
        # PolyLine hinzufügen
        folium.PolyLine(coords, color="blue", weight=2.5, opacity=1).add_to(route_map)
        
        # Optional: Punkte markieren (auskommentiert lassen, falls nicht benötigt)
        # for _, row in route_df.iterrows():
        #     folium.CircleMarker(
        #         location=[row['Latitude'], row['Longitude']],
        #         radius=2,
        #         color='red',
        #         fill=True,
        #         fill_color='red',
        #         fill_opacity=0.7,
        #         popup=f"Timestamp: {row['# Timestamp']}"
        #     ).add_to(route_map)
        
        # Karte anzeigen oder speichern
        route_map.save("ship_route.html")
        route_map

In [6]:
print(combined_df.head(10))

         MMSI   Latitude  Longitude # Timestamp
0   219000873  56.990910  10.304543  2024-03-01
2   219016683  56.800165   9.024933  2024-03-01
3   219000615  56.967093   9.224287  2024-03-01
5   219017664  56.974950   8.922530  2024-03-01
6   219002686  56.795143   8.863960  2024-03-01
7   219030053  57.058252   9.900817  2024-03-01
8   219670000  55.463782   8.444915  2024-03-01
9   211417590  54.524345  12.675237  2024-03-01
11  246846000  57.628408  10.595227  2024-03-01
12  246846000  57.628408  10.595227  2024-03-01


In [5]:
print(f"Der angepasste Datensatz hat {len(combined_df)} Einträge.")

Der angepasste Datensatz hat 29508390 Einträge.
